In [ ]:
import numpy as np
import os, cv2
import h5py
import tifffile
from tqdm import tqdm, trange
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from deepreg.predict import unwrapped_predict, normalize_batched_image
from deepreg.model.layer import Warping
import tensorflow as tf
import matplotlib.pyplot as plt


batch_size = 2
label_shape = (30, 3)
irregular = True

2025-05-20 16:33:29.766887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 16:33:29.766932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 16:33:29.769354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Register a video in batches

In [ ]:
tiff_folder = "/storage/fs/store1/brian/swimming_videos_RFa/Folder_20250212124023_RFa_noswim"

out_folder = tiff_folder

checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/multi_animal_RFa-6_fixeddp/save/ckpt-350"

max_frames = 2000
# side_len = 1080
side_len = 1024
red_chan = 1
padding = np.array([[0,0],[0,0],[0,0],[1,1]])
z_depth = 3

log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"


In [3]:
files = os.listdir(tiff_folder)
files = [f for f in files if (f.endswith(".tif") or f.endswith(".tiff")) and f.startswith("RIG_")]
files = sorted(files, key = lambda x: int(x.split("_")[-1].split("-")[0]))
print(files)

['RIG_20250212_Experiment_01_0-1999.tif']


In [4]:
inputs = tifffile.imread(os.path.join(tiff_folder, files[0]))

In [5]:
inputs.shape

(2000, 2, 1200, 1200)

In [6]:
if(len(inputs.shape) == 3):
    irregular = False
    inputs = inputs[:, np.newaxis, ...]

In [7]:
crop_shape = np.array((1024, 1024))
# crop_shape = np.array((1080, 1080))

crop_offset = (inputs.shape[2:] - crop_shape) / 2
assert np.all(crop_offset == crop_offset.astype(int))
crop_offset = crop_offset.astype(int)

## Crop the images
inputs_crop = inputs[:, :, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]
assert np.all(inputs_crop.shape[2:] == crop_shape)

In [ ]:
if irregular:
    input_red = inputs_crop[:, 1, ...]
else:
    input_red = inputs_crop[:, 0, ...]

In [9]:
if 'batched_fixed_image' not in locals():
    fixed_image_ARCHIVE = input_red[0] # So we don't have to reload stuff
    batched_fixed_image = np.repeat(np.expand_dims(input_red[0], axis=0), batch_size, axis=0).astype(np.float32)
    print("Fixed img saved")
else:
    batched_fixed_image = np.repeat(np.expand_dims(fixed_image_ARCHIVE, axis=0), batch_size, axis=0).astype(np.float32)
    print("Restored")

Fixed img saved


In [10]:
os.chdir("/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/scripts")
from register import set_GPU, register
# set_GPU(1)

In [15]:
config_path = os.path.join(checkpoint_path.split("/save")[0], "config.yaml")
model = None

red_offset = 0 # You really shouldn't need to change this, it's a rememant from when 

other_chan = (red_chan - 1) % inputs.shape[1]


batched_fixed_image_pad = np.pad(batched_fixed_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_fixed_image))

warping = Warping(fixed_image_size=(side_len, side_len, z_depth), batch_size=batch_size)


for file in files:
    inputs = tifffile.imread(os.path.join(tiff_folder, file))
    inputs_crop = inputs[:, :, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]
    input_red = inputs_crop[:, red_chan, ...]
    input_green = inputs_crop[:, other_chan, ...]
    outs = np.zeros_like(inputs_crop)
    for frame in trange(0, max_frames, batch_size):
        if frame + batch_size > (max_frames):
            frame = (max_frames) - batch_size

        batched_moving_image = input_red[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_moving_image))
        
        ddf_output, pred_fixed_image, model = unwrapped_predict(
            batched_fixed_image_pad,
            batched_moving_image,
            log_dir,
            label_shape,
            label_shape,
            model = model,
            model_ckpt_path = checkpoint_path,
            model_config_path = config_path,
        )

        gr_in = np.pad(input_green[(frame):(frame)+batch_size, ..., np.newaxis], padding, "constant", constant_values=0).astype(np.float32)
        outs[frame:(frame) + batch_size, other_chan] = warping(inputs=[ddf_output, gr_in]).numpy()[..., 1]
        
        # This is here instead of just taking `pred_fixed_image` from unwrapped_predict because that output is normalized so this is just easier to get the same scale
        batched_moving_image = input_red[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=0)
        outs[frame:(frame) + batch_size, red_chan] = warping(inputs=[ddf_output, batched_moving_image]).numpy()[..., 1]


    tifffile.imwrite(os.path.join(out_folder, "WARPED-BOTH_" + file), outs, imagej=True, bigtiff=True)



  0%|          | 0/1000 [00:00<?, ?it/s]

2025-05-20 18:26:14 | WARNING  | Log directory /home/brian/data4/brian/PBnJ/out_vids/logs exists already.
2025-05-20 18:26:14 | WARNING  | Using customized configuration. The code might break if the config doesn't match the saved model.
Built inputs.
Built control points.
Concatenated images.
{'extract_levels': ListWrapper([0, 1, 2, 3]), 'name': 'local', 'num_channel_initial': 16}
Built backbone.
Built DDF.
Built warping.


100%|██████████| 1000/1000 [1:09:36<00:00,  4.18s/it]
/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:1753: UserWarning: <tifffile.TiffWriter 'WARPED-BOTH_RIG…nt_01_0-1999.tif'> writing nonconformant BigTIFF ImageJ
  warnings.warn(


# Generate Aligned ROIs
Ideally you would generate these while you run the above section, I'm just seperating it here since I want to run it in isolation.

In [14]:
log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"

config_path = os.path.join(checkpoint_path.split("/save")[0], "config.yaml")
model = None

batched_fixed_image_pad = np.pad(batched_fixed_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_fixed_image))

warping = Warping(fixed_image_size=(side_len, side_len, z_depth), batch_size=batch_size)
ROIwarping = Warping(fixed_image_size=(side_len, side_len, z_depth), batch_size=batch_size, interpolation="nearest")

for file in files:
    inputs = tifffile.imread(os.path.join(tiff_folder, file))
    inputs_crop = inputs[:, :, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]
    input_red = inputs_crop[:, red_chan, ...]

    ROI_inputs = tifffile.imread(os.path.join(tiff_folder, file.replace("RIG_", "RIG-ROIS_")))
    ROI_inputs_crop = ROI_inputs[:, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]

    ROIouts = np.zeros_like(ROI_inputs_crop)
    for frame in trange(0, max_frames, batch_size):
        if frame + batch_size > (max_frames):
            frame = (max_frames) - batch_size
    

        batched_moving_image = input_red[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_moving_image))

        
        ddf_output, pred_fixed_image, model = unwrapped_predict(
            batched_fixed_image_pad,
            batched_moving_image,
            log_dir,
            label_shape,
            label_shape,
            model = model,
            model_ckpt_path = checkpoint_path,
            model_config_path = config_path,
        )

        ROI_in = np.pad(ROI_inputs_crop[(frame):(frame)+batch_size, ..., np.newaxis], padding, "constant", constant_values=0).astype(np.float32)
        ROIouts[frame:(frame) + batch_size] = ROIwarping(inputs=[ddf_output, ROI_in]).numpy()[..., 1]

        
    tifffile.imwrite(os.path.join(out_folder, "WARPED-ROIS_" + file), ROIouts, imagej=True, bigtiff=True)


  0%|          | 0/1000 [00:00<?, ?it/s]

2025-05-20 17:34:52 | WARNING  | Log directory /home/brian/data4/brian/PBnJ/out_vids/logs exists already.
2025-05-20 17:34:52 | WARNING  | Using customized configuration. The code might break if the config doesn't match the saved model.
Built inputs.
Built control points.
Concatenated images.
{'extract_levels': ListWrapper([0, 1, 2, 3]), 'name': 'local', 'num_channel_initial': 16}
Built backbone.
Built DDF.
Built warping.


100%|██████████| 1000/1000 [48:55<00:00,  2.94s/it] 
/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:1753: UserWarning: <tifffile.TiffWriter 'WARPED-ROIS_RIG…nt_01_0-1999.tif'> writing nonconformant BigTIFF ImageJ
  warnings.warn(
